In [1]:
import os
import sys
module_path = ".."
sys.path.append(module_path)

from dataset_preprocessing import make_keyphrase_dataset, make_bio_tagged_dataset

In [2]:
from tqdm import tqdm
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

In [3]:
MODEL_NAME = "KeyBERT"

In [4]:
def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

def process_label(text):
    processed_text = text.strip('"').replace("\n", " ").strip()
    processed_text = re.sub('\s+', ' ', processed_text)

    return processed_text

In [5]:
prediction = pd.read_csv(f"KeySRoBERTa.csv")

In [6]:
documents = prediction["본문"]
gold_standards = prediction["원인 키워드"]
model_predictions = prediction[f"{MODEL_NAME} 예측 키워드"]

In [7]:
for i in range(len(prediction)):
    prediction["원인 키워드"][i] = prediction["원인 키워드"][i]

In [8]:
model_predictions

0      "F로 잘못 인지하여(실제 Z2에서 Z3로 연결되는 부분이 F 방향과", "에는 직...
1      "지상 활주 중 속도지시계 이상으로 RAMP RETURN하여 점검", "항공기가 P...
2      "상태에서 ATC의 Left Turn SZ를 받고 지시에", "회항 과정에서 Fli...
3      "후 route set up함. 이륙 후 ANLOC 1C절차를 따라 좌선회", "A...
4      "오후에 이륙하여 일본지역의 공항으로 접근을 위해 기수(heading) 140을 유...
                             ...                        
214    "POINT RWY 02 VIA B'을 주어 차트를 확인하였다. 주기장에서 TAXI...
215    "Cockpit를 점검하는 사이에 승객은 탑승이 완료되었으나, Cockpit 준비가...
216    "했다. 여러 번 반복한 끝에 항공기는 정상 상태가", "후에야 Gear Door가...
217    "대체공항으로 회항하기로 결정했다. 관제기 관에 회항하겠다고 통보하였으나", "대체...
218    "계기비행계획으로 마닐라를 출발, 서울을 향했다. FL370에서", "우리 항공기를...
Name: KeyBERT 예측 키워드, Length: 219, dtype: object

In [9]:
gold_labels = [process_label(label) for label in prediction["원인 키워드"][0].split('", "')]
gold_labels

['Z3를 F로 잘못 인지하여', '최종 확인을 한 후 진입을 했어야 하나, 확인 미흡', '승무원 상호 간 CRM 부족']

In [10]:
gold_standard_kp_dataset = make_keyphrase_dataset(prediction, col_name="원인 키워드")
model_prediction_kp_dataset = make_keyphrase_dataset(prediction, col_name=f"{MODEL_NAME} 예측 키워드", ignore_duplication_error=True)

bio_tagged_gold_standard = make_bio_tagged_dataset(gold_standard_kp_dataset)
bio_tagged_model_prediction = make_bio_tagged_dataset(model_prediction_kp_dataset)

bio_tagged_gold_standard = bio_tagged_gold_standard.replace(to_replace="B-causal factor", value="key").replace(to_replace="I-causal factor", value="key")
bio_tagged_model_prediction = bio_tagged_model_prediction.replace(to_replace="B-causal factor", value="key").replace(to_replace="I-causal factor", value="key")

100% 219/219 [00:00<00:00, 3876.31it/s]
100% 219/219 [00:00<00:00, 4747.63it/s]
100% 219/219 [00:02<00:00, 88.21it/s] 
100% 219/219 [00:04<00:00, 52.33it/s]


In [11]:
if len(bio_tagged_gold_standard) == len(bio_tagged_model_prediction):
    print(f"Start Evaluation of {MODEL_NAME} model")
    
    print()
    
    print("precision_score")
    print(precision_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="micro"))
    print(precision_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="macro"))
    print(precision_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="weighted"))
    print(precision_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average=None))
    print(precision_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average='micro', labels=["key"]))
    
    print()
    
    print("recall_score")
    print(recall_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="micro"))
    print(recall_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="macro"))
    print(recall_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="weighted"))
    print(recall_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average=None))
    print(recall_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average='micro', labels=["key"]))
    
    print()

    print("f1_score")
    print(f1_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="micro"))
    print(f1_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="macro"))
    print(f1_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average="weighted"))
    print(f1_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average=None))
    print(f1_score(bio_tagged_gold_standard["tag"], bio_tagged_model_prediction["tag"], average='micro', labels=["key"]))

Start Evaluation of KeyBERT model

precision_score
0.6885814313606679
0.483133415526248
0.7407085706422172
[0.84679917 0.11946766]
0.1194676570721139

recall_score
0.6885814313606679
0.4769567114729473
0.6885814313606679
[0.77574587 0.17816755]
0.17816755135010384

f1_score
0.6885814313606679
0.4763729729581932
0.7124718553434753
[0.80971677 0.14302918]
0.1430291801760074
